In [1]:
import cvxpy as cp
import networkx as nx
from numpy import linalg as la
import numpy as np
import matplotlib.pyplot as plt

from joblib import Parallel, delayed
import time
import sys
from os import cpu_count
from copy import copy

import data
import opt

# CONSTANTS
N_CPUS = cpu_count()
SEED = 28

MODELS = [
    {
        "func": "estH_unpertS",
        "cy": "non-st", # not used, so not important
        "legend": "FI",
        "fmt": "o-"
    },
    {
        "func": "estH_iter",
        "cy": "non-st",
        "legend": "ITER-NONST",
        "fmt": "P-"
    },
    {
        "func": "estH_iter_rew",
        "cy": "non-st",
        "legend": "ITER-REW-NONST",
        "fmt": "p-"
    },
    # ORIGINAL
    # {
    #     "func": "estH_iter_rew",
    #     "cy": "samp",
    #     "legend": "ITER-REW-ST-SAMP",
    #     "fmt": "v-"
    # }
    {
        "func": "estH_iter_rew",
        "cy": "real",
        "legend": "ITER-REW-ST-REAL",
        "fmt": "v-"
    }]

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [2]:
def est_params(id, model, X, Y, Cy_exp, h, H, S, Sn, LAMBDS, GAMMAS, DELTAS, BETAS, INC_GAMMAS):
    K = h.size
    err_h = np.zeros((len(LAMBDS), len(GAMMAS), len(DELTAS), len(BETAS), len(INC_GAMMAS)))
    err_H = np.zeros((len(LAMBDS), len(GAMMAS), len(DELTAS), len(BETAS), len(INC_GAMMAS)))
    err_S = np.zeros((len(LAMBDS), len(GAMMAS), len(DELTAS), len(BETAS), len(INC_GAMMAS)))

    norm_h = la.norm(h)
    norm_H = la.norm(H, 'fro')
    S_normalized = S/la.norm(S, 'fro')
    for i, lambd in enumerate(LAMBDS):
        for j, gamma in enumerate(GAMMAS):
            for k, delta in enumerate(DELTAS):
                for l, beta in enumerate(BETAS):
                    for m, inc_gamma in enumerate(INC_GAMMAS):
                        if model['func'] == 'estH_iter_rew':
                            params = [lambd, gamma, delta, beta, inc_gamma]
                        else:
                            params = [lambd, gamma, delta, inc_gamma]

                        _, H_est, S_est = getattr(opt, model["func"])(X, Y, Sn, Cy_exp, params)
                        h_est = data.obtain_filter_coefs(S_est, H_est, K)

                        err_h[i,j,k,l,m] = la.norm(h - h_est)**2/norm_h**2
                        err_H[i,j,k,l,m] = la.norm(H - H_est)**2/norm_H**2
                        err_S[i,j,k,l,m] = la.norm(S_normalized - la.norm(S_est, 'fro'))**2
                        
                        if id % N_CPUS:
                            print(f'{id} - lambd: {lambd}, gamma: {gamma}, delta: {delta}, beta: {beta},', end=' ')
                            print(f'inc_gamma: {inc_gamma}, err_H: {err_H[i,j,k,l,m]:.3f}, err_S: {err_S[i,j,k,l,m]:.3f}') 
    return err_h, err_H, err_S

In [3]:
# Main
np.random.seed(SEED)
model_idx = 2

# Weights to swape
LAMBDS = [.01, .05, .1, .5, 1]
GAMMAS = [.01, .05, .1, .5, 1]
DELTAS = [0]
BETAS = [0, 1e-4, 1e-2]
INC_GAMMAS = [1, 1.1, 1.25, 1.15]

# Other params
N = 20
M = 50
p_n = .05
K = 5
n_graphs = 20
p_er = .2
# g_params = {'type': data.ER, 'p': p_er}
g_params = {'type': data.SW, 'k': 4, 'p': 0.1}
eps = .1
exp_coefs = True
decay = 1
neg_coefs = True

t = time.time()
print("CPUs used:", N_CPUS)
print("Model selected:", MODELS[model_idx], 'graph:', g_params['type'])
err_h = np.zeros((n_graphs, len(LAMBDS), len(GAMMAS), len(DELTAS), len(BETAS), len(INC_GAMMAS)))
err_H = np.zeros((n_graphs, len(LAMBDS), len(GAMMAS), len(DELTAS), len(BETAS), len(INC_GAMMAS)))
err_S = np.zeros((n_graphs, len(LAMBDS), len(GAMMAS), len(DELTAS), len(BETAS), len(INC_GAMMAS)))

funcs = []
with Parallel(n_jobs = N_CPUS) as pool:
    for i in range(n_graphs):
        X, Y, Cy, Cy_samp, H, S, Sn, h = data.gen_data(N, M, g_params, p_n, eps, K, exp_coefs=exp_coefs,
                                                       coef=decay, neg_coefs=neg_coefs)

        # Select model and covariance
        model = copy(MODELS[model_idx])
        if model["cy"] == "real":
            Cy_exp = Cy
        elif model["cy"] == "samp":
            Cy_exp = Cy_samp
        elif model["cy"] == "non-st":
            Cy_exp = np.eye(N)
        else:
            raise NotImplementedError("Choose either real or samp covariance")

        funcs.append(delayed(est_params)(i, model, X, Y, Cy_exp, h, H, S, Sn, LAMBDS, 
                                         GAMMAS, DELTAS, BETAS, INC_GAMMAS))

    results = pool(funcs)
    for i in range(n_graphs):
        err_h[i], err_H[i], err_S[i] = results[i]


CPUs used: 12
Model selected: {'func': 'estH_iter_rew', 'cy': 'non-st', 'legend': 'ITER-REW-NONST', 'fmt': 'p-'} graph: 4


In [ ]:
# Print
mean_err_h = np.mean(err_h, 0)
mean_err_H = np.mean(err_H, 0)
mean_err_S = np.mean(err_S, 0)
idx = np.unravel_index(np.argmin(mean_err_h), mean_err_h.shape)
print('Min mean err h (Lambd: {:.3g}, Gamma: {:.3g}, Delta: {:.3g}, Beta: {:.3g}, Inc: {:.3g}): {:.6f})'
        .format(LAMBDS[idx[0]], GAMMAS[idx[1]], DELTAS[idx[2]], BETAS[idx[3]], INC_GAMMAS[idx[4]], mean_err_h[idx]))

idx = np.unravel_index(np.argmin(mean_err_H), mean_err_H.shape)
print('Min mean err H (Lambd: {:.3g}, Gamma: {:.3g}, Delta: {:.3g}, Beta: {:.3g}, Inc: {:.3g}): {:.6f})'
        .format(LAMBDS[idx[0]], GAMMAS[idx[1]], DELTAS[idx[2]], BETAS[idx[3]], INC_GAMMAS[idx[4]], mean_err_h[idx]))

idx = np.unravel_index(np.argmin(mean_err_S), mean_err_S.shape)
print('Min mean err S (Lambd: {:.3g}, Gamma: {:.3g}, Delta: {:.3g}, Beta: {:.3g}, Inc: {:.3g}): {:.6f})'
        .format(LAMBDS[idx[0]], GAMMAS[idx[1]], DELTAS[idx[2]], BETAS[idx[3]], INC_GAMMAS[idx[4]], mean_err_h[idx]))

# med_err_h = np.median(err_h, 0)
# idx = np.unravel_index(np.argmin(med_err_h), med_err_h.shape)
# print('Min median err (Lambd: {:.3g}, Gamma: {:.3g}, Delta: {:.3g}, Beta: {:.3g}, Inc: {:.3g}): {:.6f})'
#         .format(LAMBDS[idx[0]], GAMMAS[idx[1]], DELTAS[idx[2]], BETAS[idx[3]], INC_GAMMAS[idx[4]], med_err_h[idx]))

Min mean err (Lambd: 0.02, Gamma: 0.5, Delta: 0, Beta: 0, Inc: 1): 1.399827)
Min median err (Lambd: 0.02, Gamma: 0.5, Delta: 0, Beta: 0, Inc: 1): 0.011083)
